[View in Colaboratory](https://colab.research.google.com/github/mancap314/text_generation/blob/master/text_generation_trung.ipynb)

# LSTM Text Generation 

Based on this [blog post](https://chunml.github.io/ChunML.github.io/project/Creating-Text-Generator-Using-Recurrent-Neural-Network/) by Trung Tran

## Step 1: Import Text Data

We just import *War and Peace* by Tolstoy (hopefully the machine can read it faster than I...)

In [2]:
!wget https://cs.stanford.edu/people/karpathy/char-rnn/warpeace_input.txt

--2018-06-28 00:12:26--  https://cs.stanford.edu/people/karpathy/char-rnn/warpeace_input.txt
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3258246 (3.1M) [text/plain]
Saving to: ‘warpeace_input.txt.2’

warpeace_input.txt. 100%[===================>]   3.11M  --.-KB/s    in 0.1s    

2018-06-28 00:12:27 (20.8 MB/s) - ‘warpeace_input.txt.2’ saved [3258246/3258246]



## Step 2: Prepare the data

First read the data and get the set of unique characters:



In [3]:
DATA_PATH = 'warpeace_input.txt'

data = open(DATA_PATH, 'r').read()
chars = list(set(data)) #set: gets unique values
VOCAB_SIZE = len(chars)

print('chars:\n{}\n\nVOCAB_SIZE: {}'.format(chars, VOCAB_SIZE))

chars:
['(', 'c', '!', 'Y', 'R', 'Q', 'r', '4', 'i', 'é', 'J', 'x', 't', 'v', ':', 'l', '-', 'e', '2', ' ', 'B', "'", 'N', 'E', 'z', 'C', '*', '\n', 'à', 'U', 'ä', 'O', '1', 'k', 'D', '\ufeff', 'Z', 'd', 'V', 'o', 'p', 'I', 'm', ')', 'A', ';', 'S', 'q', 'X', 'b', '7', '9', '/', 'y', 'L', 'P', 'a', 'g', 'H', '5', 's', 'f', ',', 'G', 'K', '.', '0', '?', 'M', 'ê', '3', '6', 'w', '=', '"', 'j', 'h', 'W', 'T', 'u', '8', 'F', 'n']

VOCAB_SIZE: 83


Then map the characters to integer numbers (and vice-versa):

In [4]:
idx_to_char = {i: char for i, char in enumerate(chars)}
char_to_idx = {char: i for i, char in enumerate(chars)}

Now let prepare the ground for the feature data **X** and the target variable **y**:

In [5]:
import numpy as np

SEQ_LENGTH = 60 #input sequence length
N_FEATURES = VOCAB_SIZE #one hot encoding here, that's why, but deduplicated for clarity

N_SEQ = int(np.floor((len(data) - 1) / SEQ_LENGTH))

X = np.zeros((N_SEQ, SEQ_LENGTH, N_FEATURES))
y = np.zeros((N_SEQ, SEQ_LENGTH, N_FEATURES))

for i in range(N_SEQ):
  X_sequence = data[i * SEQ_LENGTH: (i + 1) * SEQ_LENGTH]
  X_sequence_ix = [char_to_idx[c] for c in X_sequence]
  input_sequence = np.zeros((SEQ_LENGTH, N_FEATURES))
  for j in range(SEQ_LENGTH):
    input_sequence[j][X_sequence_ix[j]] = 1. #one-hot encoding of the input characters
  X[i] = input_sequence
  
  y_sequence = data[i * SEQ_LENGTH + 1: (i + 1) * SEQ_LENGTH + 1] #shifted by 1 to the right
  y_sequence_ix = [char_to_idx[c] for c in y_sequence]
  target_sequence = np.zeros((SEQ_LENGTH, N_FEATURES))
  for j in range(SEQ_LENGTH):
    target_sequence[j][y_sequence_ix[j]] = 1. #one-hot encoding of the target characters
  y[i] = target_sequence

## Step 3: define the model:

In [6]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, TimeDistributed, Dense, Activation

# constant parameter for the model
HIDDEN_DIM = 700 #size of each hidden layer, "each layer has 700 hidden states"
LAYER_NUM = 2 #number of hidden layers, how much were used?
NB_EPOCHS = 200 #max number of epochs to train, "200 epochs"
BATCH_SIZE = 128 
VALIDATION_SPLIT = 0.1 #proportion of the batch used for validation at each epoch


model = Sequential()
model.add(CuDNNLSTM(HIDDEN_DIM, 
               input_shape=(None, VOCAB_SIZE), 
               return_sequences=True))
for _ in range(LAYER_NUM - 1):
  model.add(CuDNNLSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

Using TensorFlow backend.


Create function to generate text from the trained model:

In [7]:
def generate_text(model, length):
  ix = [np.random.randint(VOCAB_SIZE)]
  y_char = [idx_to_char[ix[-1]]]
  X = np.zeros((1, length, VOCAB_SIZE))
  for i in range(length):
    X[0, i, :][ix[-1]] = 1.
    ix = np.argmax(model.predict(X[:, :i+1,:])[0], 1)
    y_char.append(idx_to_char[ix[-1]])
  return ''.join(y_char)

Define callbacks:

In [8]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
# callback to save the model if better
filepath="tgt_model.hdf5"
save_model_cb = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# callback to stop the training if no improvement
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=0)
# callback to generate text at epoch end
class generateText(Callback):
    def on_epoch_end(self, batch, logs={}):
        print(generate_text(self.model, 100))
        
generate_text_cb = generateText()

callbacks_list = [save_model_cb, early_stopping_cb, generate_text_cb]

## Step 4: run the model:

In [0]:

model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=NB_EPOCHS, callbacks=callbacks_list, validation_split=VALIDATION_SPLIT)

Train on 47943 samples, validate on 5327 samples
Epoch 1/200
47943/47943 [==============================] - 80s 2ms/step - loss: 2.5003 - acc: 0.3016 - val_loss: 1.9147 - val_acc: 0.4395

Epoch 00001: val_acc improved from -inf to 0.43945, saving model to tgt_model.hdf5
-
the stold the stold the stold the stold the stold the stold the stold the stold the stold the stold
Epoch 2/200
 7936/47943 [===>..........................] - ETA: 1:02 - loss: 1.8448 - acc: 0.4568

47943/47943 [==============================] - 78s 2ms/step - loss: 1.6546 - acc: 0.5094 - val_loss: 1.5094 - val_acc: 0.5463

Epoch 00002: val_acc improved from 0.43945 to 0.54632, saving model to tgt_model.hdf5
5 the first the first the first the first the first the first the first the first the first the first
Epoch 3/200
13312/47943 [=======>......................] - ETA: 54s - loss: 1.4413 - acc: 0.5682